In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from os.path import join as pjoin
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from scipy.stats import pearsonr, spearmanr, zscore
import itertools

sys.path.append('/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis')
import circletrack_behavior as ctb
import circletrack_neural as ctn
import place_cells as pc
import plotting_functions as pf
import cell_overlap as co

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
## Settings
dpath = '../../../MultiCon_Imaging/MultiCon_Imaging4/output/'
fig_path = '../../../MultiCon_Imaging/MultiCon_Imaging4/intermediate_figures/'
bin_size = 0.4
velocity_thresh = 7
mouse_list = ['mc42', 'mc43']
session_list = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]

In [6]:
## Cell overlap across days
## Distance of 4 pixels between cell centers
centroid_distance = 4
mouse = 'mc42'
fig = pf.custom_graph_template(x_title='Day', y_title='', width=600, 
                               shared_y=True, titles=[mouse])
mappings = pd.read_pickle(pjoin(dpath, f'cross_registration_results/circletrack_data/{mouse}/mappings_{centroid_distance}.pkl'))
overlap = co.calculate_overlap(mappings)
overlap = co.dates_to_days(overlap, '2024_06_12', days=20)
matrix = overlap.pivot_table(index='session_id1', columns='session_id2', values='overlap')
fig.add_trace(go.Heatmap(z=matrix.values, x=matrix.index, y=matrix.columns, coloraxis='coloraxis'))
boundaries = [5.5, 10.5, 15.5]
for boundary in boundaries:
        fig.add_vline(x=boundary, line_width=1.5, line_color='red', opacity=1)
        fig.add_hline(y=boundary, line_width=1.5, line_color='red', opacity=1) 
fig.update_yaxes(title='Day', col=1)
fig.update_layout(coloraxis_colorbar={'title': 'Overlap (%)'})
fig.show()
fig.write_image(pjoin(fig_path, f'{mouse}_overlap_heatmaps.png'))